## - - 이전 코드 시작 - -

In [1]:
#의존성 패키지 설치
!pip install ratsnlp

     |████████████████████████████████| 42 kB 562 kB/s 
     |████████████████████████████████| 57 kB 4.9 MB/s 
     |████████████████████████████████| 806 kB 35.3 MB/s 
     |████████████████████████████████| 2.8 MB 31.7 MB/s 
     |████████████████████████████████| 636 kB 42.2 MB/s 
     |████████████████████████████████| 136 kB 36.1 MB/s 
     |████████████████████████████████| 829 kB 34.2 MB/s 
     |████████████████████████████████| 398 kB 44.1 MB/s 
     |████████████████████████████████| 77 kB 6.4 MB/s 
     |████████████████████████████████| 895 kB 32.9 MB/s 
     |████████████████████████████████| 3.3 MB 36.9 MB/s 
     |████████████████████████████████| 1.1 MB 33.6 MB/s 
     |████████████████████████████████| 96 kB 5.6 MB/s 
     |████████████████████████████████| 144 kB 12.2 MB/s 
     |████████████████████████████████| 271 kB 26.7 MB/s 
     |████████████████████████████████| 94 kB 3.4 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sh

In [3]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [4]:
#모델 환경 설정
import torch
from ratsnlp.nlpbook.generation import GenerationTrainArguments
args = GenerationTrainArguments(
    pretrained_model_name="skt/kogpt2-base-v2",
    downstream_corpus_name="nsmc",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-generation",
    max_seq_length=32,
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs=3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

In [5]:
#랜덤 시드 고정
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


In [6]:
#로거 설정
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters GenerationTrainArguments(pretrained_model_name='skt/kogpt2-base-v2', downstream_task_name='sentence-generation', downstream_corpus_name='nsmc', downstream_corpus_root_dir='/content/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-generation', max_seq_length=32, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, cpu_workers=2, fp16=False, tpu_cores=0)


In [7]:
#말뭉치 다운로드
from Korpora import Korpora
Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
    force_download=args.force_download,
)

[nsmc] download ratings_train.txt: 14.6MB [00:00, 86.3MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 38.8MB/s]                            


In [8]:
#토크나이저 준비
from transformers import PreTrainedTokenizerFast
tokenizer = PreTrainedTokenizerFast.from_pretrained(
    args.pretrained_model_name,
    eos_token="</s>",
)

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


## - - 이전 코드 끝 - -

<**5단계**> **데이터 전처리하기**   
딥러닝 모델을 학습하려면 데이터 로더를 이용해 데이터를 배치 단위로 모델에 공급해 주어야 합니다.   
다음 코드를 실행하면 학습용 데이터 로더의 구성 요소인 데이터셋(`train_dataset`)이 만들어집니다.   

In [9]:
#학습 데이터셋 구축
from ratsnlp.nlpbook.generation import NsmcCorpus, GenerationDataset
corpus = NsmcCorpus()
train_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading train data... LOOKING AT /content/Korpora/nsmc/ratings_train.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 9.630 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 부정 아 더빙.. 진짜 짜증나네요 목소리
INFO:ratsnlp:tokens: ▁부정 ▁아 ▁더 빙 .. ▁진짜 ▁짜 증 나 네 요 ▁목소리 </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
INFO:ratsnlp:features: GenerationFeatures(input_ids=[11775, 9050, 9267, 7700, 9705, 23971, 12870, 8262, 7055, 7098, 8084, 48213, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], labels=[11775, 9050, 9267, 7700, 9705, 23971, 12870, 8262, 7055, 7098, 8084, 48213

아무런 전처리를 하지 않은 NSMC 데이터는 다음처럼 생겼습니다. `id`는 영화 리뷰의 아이디, `document`는 영화 리뷰 문장, `label`은 평점을 가공해 만든 극성 레이블입니다. `0`은 부정, `1`은 긍정이라는 의미입니다. 참고로 이와 같은 NSMC 데이터를 읽어들이는 역할을 하는 `NsmcCorpus`는 <4-2>절 문서 분류 모델을 실습할 때 사용한 클래스와 동일합니다.


|id|document|label|
|:---:|:---:|:---:|
|9976970|아 더빙.. 진짜 짜증나네요 목소리|0|
|3819312|흠..포스터보고 초딩영화줄....오버연기조차 가볍지 않구나|0|     
|10265843|너무재밓었다그래서보는것을추천한다|1|   

   
<center><표 - NSMC 데이터 예></center>    

우리가 파인튜닝할 모델은 텍스트 왼쪽부터 오른쪽으로 순차적으로 읽으면서 학습하는 GPT 모델입니다. 파인튜닝 시 문장 맨 앞에 극성(polarity) 정보를 부여합니다. 이로써 인퍼런스 과정에서 임의의 극성 정보를 주었을 때 해당 극성에 맞는 문장을 생성할 수 있는, ***조건부 문장 생성(conditional text generation)*** 능력이 있는지를 검증하고자 합니다.

- **부정** 아 더빙.. 진짜 짜증나네요 목소리
- **부정** 흠..포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
- **긍정** 너무재밓었다그래서보는것을추천한다  

cf) 볼드체 = 극성 정보

`GenerationDataset` 클래스는 `NsmcCorpus`와 4단계에서 선언해 둔 토크나이저를 품고 있는데요, 이 클래스는 `NsmcCorpus`가 넘겨준 문장과 극성을 모델이 학습할 수 있는 형태(`GenerationDataset`)로 가공합니다.  
 `GenerationDataset`라는 자료형에는 다음과 같은 네 가지 정보가 있습니다.

- **input_ids**: List[int]
- **attention_mask**: List[int]
- **token_type_ids**: List[int]
- **labels**: List[int]

그러면 `NsmcCorpus`가 넘겨준 0번 데이터를 살펴봅시다. 여기서 레이블 `0`은 부정이라는 뜻입니다.

- **text**: 아 더빙.. 진짜 짜증나네요 목소리
- **label**: 0

코랩에서 5단계를 실행한 뒤 `train_dataset[0]`을 입력하면 다음과 같은 결과를 확인할 수 있습니다. `NsmcCorpus`가 넘겨준 0번 데이터(**아 더빙.. 진짜 짜증나네요 목소리, 0**)가 `GenerationFeatures`라는 0번 인스턴스로 변환된 것입니다. 이처럼 `GenerationDataset`이 가지고 있는 모든 인스턴스는 인덱스로 접근할 수 있습니다.

In [10]:
train_dataset[0]

GenerationFeatures(input_ids=[11775, 9050, 9267, 7700, 9705, 23971, 12870, 8262, 7055, 7098, 8084, 48213, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], labels=[11775, 9050, 9267, 7700, 9705, 23971, 12870, 8262, 7055, 7098, 8084, 48213, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

`input_ids`는 극성 정보가 포함된 입력 문장(**부정 아 더빙.. 진짜 짜증나네요 목소리**)을 토큰화한 뒤 이를 인덱싱한 결과입니다. `input_ids`는 뒤 쪽에 `1`이 많이 붙어 있음을 확인할 수 있습니다. `1`은 이전 코드에서 정의한 `eos_token`(`</s>`)에 대응하며 패딩 역할을 합니다.

분석 대상 문장의 토큰 길이가 앞서 정의한 `max_seq_length`보다 짧아서 패딩한 것입니다. 이보다 긴 문장일 때는 시퀀스 길이를 32로 줄입니다. `attention_mask`는 해당 토큰이 패딩 토큰인지(`0`) 아닌지(`1`)를 나타내며, 전체가 동일한 문서이기 때문에 세그먼트 정보인 `token_type_ids`는 모두 `0`을 넣습니다.

`labels`는 `input_ids`와 같습니다. 잠시 후 코드에서 사용할 `GPT2LMHeadModel`이 파인튜닝 과정에서 `labels`를 오른쪽으로 한 칸씩 옮겨서 파인튜닝 태스크가 '입력 토큰의 다음 토큰 맞히기'가 되도록 합니다. `train_dataset[0]`을 예로 들면 다음 표와 같습니다. 이후 모델의 토큰별 확률 분포에서 `labels`에 해당하는 확률은 높이고 이외의 토큰은 낮추는 방식으로 모델 전체를 업데이트하는 것이 파인튜닝 과정입니다.

|항목|토큰0|토큰1|토큰2|토큰3|토큰4|⋯|토큰10|토큰11|토큰12|⋯|
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
|input_ids|11775|9050|9267|7700|9705|⋯|8084|48213|1|⋯|
|labels|9050|9267|7700|9705|23971|⋯|48213|1|1|⋯|  

<center><표 - GPT2LMHeadModel에서의 모델 입출력></center>  

이렇게 학습 데이터셋(`train_dataset`)을 구축했으면 다음과 같은 학습용 데이터 로더를 만듭니다.

In [11]:
#학습 데이터 로더 구축
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

그리고 다음 코드를 실행해서 평가용 데이터 로더를 구축합니다.

In [12]:
#평가용 데이터 로더 구축
from torch.utils.data import SequentialSampler
val_dataset = GenerationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Creating features from dataset file at /content/Korpora/nsmc
INFO:ratsnlp:loading test data... LOOKING AT /content/Korpora/nsmc/ratings_test.txt
INFO:ratsnlp:tokenize sentences, it could take a lot of time...
INFO:ratsnlp:tokenize sentences [took 3.843 s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:sentence: 긍정 굳 ㅋ
INFO:ratsnlp:tokens: ▁긍정 ▁굳 ▁ ᄏ </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s> </s>
INFO:ratsnlp:features: GenerationFeatures(input_ids=[16420, 12969, 739, 605, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], attention_mask=[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], labels=[16420, 12969, 739, 605, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

<**6단계**> **프리트레인 마친 모델 읽어들이기**   

다음 코드를 수행해 모델을 초기화합니다. `GPT2LMHeadModel`은 KoGPT2가 프리트레인할 때 썼던 모델 클래스입니다. 앞에서 살펴본 것처럼 이번 과제는 프리트레인 태스크와 파인튜닝 태스크가 '다음 단어 맞히기'로 같기 때문에 똑같은 모델 클래스를 사용합니다.

In [13]:
#모델 초기화
from transformers import GPT2LMHeadModel
model = GPT2LMHeadModel.from_pretrained(
    args.pretrained_model_name,
)

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

<**7단계**> **모델 파인튜닝하기**    

파이토치 라이트닝이 제공하는 `lightning` 모듈을 상속받아 모델과 옵티마이저, 학습 과정 등이 정의된 태스크를 정의합니다. 다음 코드를 실행하면 문장 생성용 태스크(task)를 정의할 수 있습니다. 모델은 바로 앞 코드에서 준비한 모델 클래스를 사용하고, 옵티마이저로는 아담(Adam)을 사용합니다.

In [14]:
#태스크 정의
from ratsnlp.nlpbook.generation import GenerationTask
task = GenerationTask(model, args)

마지막으로 GPU/TPU 설정, 로그와 체크포인트 등의 설정을 알아서 해주는 트레이너를 정의하고 이 트레이너의 `fit()` 함수를 호출해 학습을 시작합니다.

In [15]:
#트레이너 정의
trainer = nlpbook.get_trainer(args)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: Checkpoint directory /gdrive/My Drive/nlpbook/checkpoint-generation exists and is not empty.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [16]:
#학습 개시
trainer.fit(
    task,
    train_dataloader=train_dataloader,
    val_dataloaders=val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 125 M 
------------------------------------------
125 M     Trainable params
0         Non-trainable params
125 M     Total params
500.656   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

#### 맺음말   

이번 절에서는 문장 생성 모델 파인튜닝을 실습해 봤습니다. NSMC를 가지고 파인튜닝했으므로 이 모델은 '극성 정보가 주어졌을 때 해당 극성에 맞는 영화 감상평'을 생성할 수 있습니다.